In [1]:
%%capture
! pip install pyLDAvis
! pip install pymorphy2

In [14]:
from gensim.test.utils import common_corpus
from gensim.models import LdaSeqModel
import gensim
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import warnings
import json
import pymorphy2
import nltk
import re 
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import nltk
from nltk import ngrams
import time
import dateutil.parser as dp
nltk.download('stopwords')
import pandas as pd
from sklearn.model_selection import train_test_split
import datetime
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from typing import List
import numpy as np

russian_stopwords = stopwords.words("russian")
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>


In [23]:
def extract_keywords(tokens: List[str],
                     embeddings: KeyedVectors,
                     top_n: int = 5,
                     diversity: float = 0.5,
                     nr_candidates: int = 20) -> str:
    candidates = []
    doc_embedding = text2vec(tokens, embeddings).reshape(1, -1)
    candidate_embeddings = []
    for candidate in tokens:
        if candidate in embeddings:
            candidates.append(candidate)
            candidate_embeddings.append(embeddings[candidate])

    if len(candidates) == 0:
        return tokens

    # Calculate distances and extract keywords
    distances = cosine_similarity(np.nan_to_num(doc_embedding), np.nan_to_num(candidate_embeddings))
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]][::-1]

    return list(set(keywords))

def text2vec(tokens: List[str], embeddings: KeyedVectors) -> np.ndarray:
    relevant = 0
    words_vecs = np.zeros((embeddings.vector_size,))
    for word in tokens:
        if word in embeddings:
            words_vecs += embeddings.word_vec(word, True)
            relevant += 1

    if relevant:
        words_vecs /= relevant
    return words_vecs

In [24]:
row_data = pd.read_json("/home/jovyan/work/outputs.jsonl", lines = True, orient = 'records')

In [25]:
row_data

,link,text,title,source,timestamp,refers_to
0,https://www.garant.ru/news/1569973/,© ilixe48 / Фотобанк Фотодженика В этом матери...,Частичная мобилизация – 2022: трудовые права и...,garant,1665171180000,[https://www.garant.ru/ia/aggregator/?tag_id=2...
1,https://www.garant.ru/news/1578551/,© sonerbakir / Фотобанк Фотодженика Что измени...,Минприроды России подготовило разъяснения о по...,garant,1665170700000,"[https://base.garant.ru/77151921/, https://bas..."
2,https://www.garant.ru/news/1578549/,© hypnotype / Фотобанк Фотодженика Необходимо ...,Утверждены размеры платы за предоставление све...,garant,1665169920000,[https://premier.garantexpress.ru/premiergaran...
3,https://www.garant.ru/news/1578550/,© AndreyPopov / Фотобанк Фотодженика С 22 сент...,Утверждена формула для расчета платы за технол...,garant,1665169260000,[https://base.garant.ru/405378469/]
4,https://www.garant.ru/news/1578544/,"© ilze79 / Фотобанк Фотодженика АО ""Эксплуатир...",Президент РФ подписал указ об особенностях пра...,garant,1665167580000,[https://base.garant.ru/405385011/]
...,...,...,...,...,...,...
119223,https://www.banki.ru/news/lenta/?id=10963284,Уполномоченный при президенте России по защите...,Титов предложил ограничить ставку эквайринга п...,BANKIRU,1647945660000,[]
119224,https://www.banki.ru/news/lenta/?id=10963307,"Крупнейший в мире контейнерный перевозчик, дат...",Датская компания-перевозчик Maersk заявила о п...,BANKIRU,1647959100000,[]
119225,https://www.banki.ru/news/lenta/?id=10963286,Министерство финансов России предложило смягчи...,Минфин предложил втрое снизить штрафы за незак...,BANKIRU,1647949800000,[]
119226,https://www.banki.ru/news/lenta/?id=10963295,В России планируют создать единое облако для р...,Правительство предложит компаниям перейти на и...,BANKIRU,1647952380000,[https://www.kommersant.ru/doc/5270264?query=%...


In [26]:
class Preparator():
    def __init__(self):
        self.morph = pymorphy2.MorphAnalyzer()
        self.tokenizer = nltk.WordPunctTokenizer()
        self.regexp = re.compile('[-%-,_\.\n\\\t\r/\W]')  # оставляет только буквы и цифры
        self.pos_decoder = {"ADJF": "ADJ",
                            'ADJS': 'ADJ',
                            'COMP': 'ADJ',
                            'INFN': 'VERB',
                            'PRTF': 'VERB'}

    def norm_word(self, word: str) -> str:
        w = str(re.sub(' +', ' ', self.regexp.sub(' ', word.lower()))).strip()
        if len(w) > 1 and w[0].isalpha():
            parsed = self.morph.parse(w)[0]
            tag = str(parsed.tag.POS)
            if tag in ['NOUN', 'VERB', 'ADJ']:
                tag = tag if tag not in self.pos_decoder else self.pos_decoder[tag]
                return parsed.normal_form.replace('ё', 'e')
        return ''
    
    def norm_sentence(self, sentence: str):
        tokens = list(filter(lambda x: len(x), [self.norm_word(word) for word in self.tokenizer.tokenize(sentence.lower())]))
        # bigrams = ['_'.join(w) for w in  ngrams(tokens,n=2)]
        return tokens
    
        

In [27]:
W2V_SOURCE = "/home/jovyan/work/custom_word2vec_uncod.bin" # Путь к файлу custom_word2vec_uncod.bin
prep = Preparator()
w2v = KeyedVectors.load_word2vec_format(
            W2V_SOURCE, binary=True, unicode_errors="ignore"
        )
texts = [extract_keywords(prep.norm_sentence(i), w2v) if i else [] for i in tqdm(row_data['title'])]

  0%|          | 0/119228 [00:00<?, ?it/s]

/opt/conda/lib/python3.9/site-packages/numpy/linalg/linalg.py:2556: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/opt/conda/lib/python3.9/site-packages/numpy/linalg/linalg.py:2556: RuntimeWarning: invalid value encountered in multiply
  s = (x.conj() * x).real
/opt/conda/lib/python3.9/site-packages/gensim/models/keyedvectors.py:450: RuntimeWarning: invalid value encountered in divide
  result = self.vectors[index] / self.norms[index]
/opt/conda/lib/python3.9/site-packages/gensim/models/keyedvectors.py:450: RuntimeWarning: invalid value encountered in divide
  result = self.vectors[index] / self.norms[index]
/opt/conda/lib/python3.9/site-packages/gensim/models/keyedvectors.py:450: RuntimeWarning: invalid value encountered in divide
  result = self.vectors[index] / self.norms[index]
/opt/conda/lib/python3.9/site-packages/gensim/models/keyedvectors.py:450: RuntimeWarning: invalid value encountered in divide
  result = self.vectors[index] / self.norms[index]
/

In [31]:
texts[0]

['гарантия', 'гражданин', 'право']

In [37]:
row_data['short'] = texts
row_data.to_json("/home/jovyan/work/outputs_upd.jsonl", lines = True, orient = 'records')

In [32]:
row_data.dtypes

link         object
text         object
title        object
source       object
timestamp     int64
refers_to    object
dtype: object

In [33]:
list(row_data.timestamp)[0]

1665171180000

In [34]:
min(row_data.timestamp), max(row_data.timestamp)

(-6795364578871, 1665181237000)

In [35]:
row_data['normed_text'] = texts
row_data['slice'] = [datetime.datetime.utcfromtimestamp(ts//1000).year for ts in row_data['timestamp']]
train, test = train_test_split(row_data, test_size=0.7, random_state=42, shuffle = True)

In [36]:
row_data.slice.value_counts()

2022    32376
2018    12735
2021    10610
2019    10352
2020     9473
2017     9171
2016     7527
2015     5419
2008     4302
2009     3412
2011     3378
2010     3134
2014     2570
2012     2188
2013     2041
2007      536
1754        4
Name: slice, dtype: int64

In [25]:
list(test.timestamp)[0]

1296739500000

In [26]:
dictionary = gensim.corpora.Dictionary(train.normed_text)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in train.normed_text]
time_slice = train.slice
num_topics = 20

In [27]:
dictionary.num_pos, dictionary.num_docs, dictionary.num_nnz

(310298, 35768, 308721)

In [ ]:
%%time
ldaseq = LdaSeqModel(corpus=doc_term_matrix, id2word=dictionary, time_slice=time_slice, num_topics=num_topics)
with open('/home/jovyan/work/ldaseq2.pkl', 'wb') as f:
     pickle.dump(ldaseq, f)